In [ ]:
import sys, os
import subprocess
print(sys.version)

In [ ]:
import pandas
import numpy
import matplotlib.pyplot as pyplot
%matplotlib inline

In [ ]:
# There are issues in some ipython versions with this
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()

logger.debug('debug logging enabled')

In [ ]:
# parameters
trace_dat_file = 'trace.dat'

In [ ]:
print('Using trace file {}'.format(os.path.realpath(trace_dat_file)))
from idlatest import estimate_idle_latency
df = estimate_idle_latency(trace_dat_file)
df['wake_lat'] = df['exit_ts'] - df['wake_ts']

In [ ]:
df

In [ ]:
def format_sec(val, tn):
    """Format real number of seconds as ns/us/ms with 3 digits"""
    if val < 0.000000900:
        return '%.3gns' % (val * 1000000000)
    if val < 0.000900000:
        return '%.3gus' % (val * 1000000)
    if val < 0.900000000:
        return '%.3gms' % (val * 1000)
    else:
        return '%.3gs' % (val)

def plot_hist(df, title='Idle Latency Plot'):
    lat = df['wake_lat']
    lat = lat[lat.notnull()]
    if lat.empty:
        return
    
    pyplot.figure(figsize=(16, 4))
    
    histcnts, histbins, _ = pyplot.hist(lat, bins=64, align='left')
    
    pyplot.title(title)
    pyplot.ylabel('Events')
    pyplot.xlabel('Latency')
    pyplot.xticks(histbins, rotation=70)
    axes = pyplot.axes()
    axes.xaxis.set_major_formatter(pyplot.FuncFormatter(format_sec))
    from matplotlib.ticker import MaxNLocator
    axes.yaxis.set_major_locator(MaxNLocator(integer=True))
    pyplot.show()

## Count events per-cpu

In [ ]:
df.groupby(['cpu']).size().reset_index(name='counts')

In [ ]:
pandas.crosstab(df['cpu'], df['state'])

## Report on a per-state basis

In [ ]:
plot_hist(df.query('state == 0'), title="state0")
plot_hist(df.query('state == 1'), title="state1")
plot_hist(df.query('state == 2'), title="state2")
plot_hist(df, title="all")

In [ ]:
df.query("cpu == 1").sort_values(['wake_lat'], ascending=False)